In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
import matplotlib.style as style
import plotly.express as px
import seaborn as sns
from pandas.plotting import scatter_matrix
from sklearn.decomposition import PCA
from sklearn import linear_model, decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier



In [ ]:
df=pd.read_csv(r'/kaggle/input/heart-disease-uci/heart.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.describe(include="all").T

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

1. ###### We can see there is no missing values

In [ ]:
df.groupby('target').size()

#### Taget variable seems quite balanced.

In [ ]:
df.plot(subplots=True, sharex=True ,figsize=(20,50))
plt.show()

## Data Visualization

In [ ]:
df.corr()['target'].sort_values(ascending=False)

In [ ]:
style.use("ggplot")
sns.set_style('whitegrid')
mask = np.zeros_like(df.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
plt.subplots(figsize = (16,9))

sns.heatmap(df.corr(),annot=True,mask=mask)
plt.show()

In [ ]:
df.drop('target', axis=1).corrwith(df.target).plot(kind='bar', grid=True, figsize=(12, 8), color=['salmon'],
                                                   title="Correlation with target")

### Correlation
**As we can see chol and fbs are very least correlated with target.**

In [ ]:
df['sex'].value_counts().plot(kind='barh',color=['r','y'])

In [ ]:
style.use("ggplot")
plt.figure(figsize=(16,9))
pd.crosstab(df.target, df.sex).plot(kind='bar')
plt.xlabel("0-No disease 1-Suffering from Disease")
plt.title("Male and Female")
plt.legend(["F","M"])
plt.show()

## Obserbvation
We can observe that female population has less suffering than male.

In [ ]:
dis_var=[]
con_var=[]
for i in df.columns:
    if len(df[i].unique())>10:
        con_var.append(i)
    else:
        dis_var.append(i)
print("Categeorical variable: ",dis_var)
print("Continuous variable: ",con_var)

In [ ]:
plt.figure(figsize=(16,16))
for i,j in enumerate(dis_var):
    plt.subplot(3,3,i+1)
    df[df["target"] == 0][j].hist(bins=35, color='blue', label='Have Heart Disease = NO', alpha=0.8)
    df[df["target"] == 1][j].hist(bins=35, color='red', label='Have Heart Disease = YES', alpha=0.8)
    plt.xlabel(j,fontsize=16)
    plt.legend(fontsize=13)

## Observation
#### cp(chest-pain)
(Value 0: typical angina
Value 1: atypical angina
Value 2: non-anginal pain
Value 3: asymptomatic)
We can see paitient with value 1 2 3 are more likely to have heart disease than value 0(typical angina)

#### restecg: (resting electrocardiographic results)
People with value 1 is more likely to have heart disease

#### exang(whether the patient had angina during exercise)

Val 0: no
Val 1: yes
people with a value of 0 (No ==> angina induced by exercise) have more heart disease than people with a value of 1

#### slope(slope of the ST segment during the most demanding part of the exercise)
People with a slope value of 2 (Downslopins: signs of an unhealthy heart) are more likely to have heart disease than people
with a slope value of 0(Upsloping: best heart rate with exercise) 
or 1 (Flatsloping: minimal change (typical healthy heart)).

#### thal(Results of the blood flow observed via the radioactive dye.
People with a thal value of 2 (defect corrected: once was a defect but ok now) are more likely to have heart disease.

#### ca(Number of main blood vessels coloured by the radioactive dye)
people with value 0 are more likely to suffer from heart disease

In [ ]:
plt.figure(figsize=(16,16))
for i,j in enumerate(con_var):
    plt.subplot(3,3,i+1)
    df[df["target"] == 0][j].hist(bins=35, color='blue', label='Have Heart Disease = NO', alpha=0.8)
    df[df["target"] == 1][j].hist(bins=35, color='red', label='Have Heart Disease = YES', alpha=0.8)
    plt.xlabel(j,fontsize=16)
    plt.legend(fontsize=13)

## Observation
#### age 
    patient age ranging between 40 to 60 are prone to heart disease

#### trestbps(resting blood pressure in millimeters of mercury (mm Hg) when the patient was admitted to the hospital)
    danger is more between 130mm hg to 140mm Hg

#### chol(cholestrol)
    above 200 is at higher risk

#### thalach(max heart rate during the stress test)
    patient with max 140 are at higher risk


In [ ]:
plt.figure(figsize=(10, 8))

plt.scatter(df.age[df.target==1],
            df.thalach[df.target==1],
            c="red")

plt.scatter(df.age[df.target==0],
            df.thalach[df.target==0],
            c="lightblue")
plt.title("Age and Max Heart Rate")
plt.xlabel("age")
plt.ylabel("max Heart Rate")
plt.legend(["Suffering:yes", "Suffering:no"]);

## Observation
we can see many young patient has higher max heart rate yet they are safe as compared to old people

## Data Preprocessing

In [ ]:
dataset = pd.get_dummies(df, columns=['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal'], drop_first=True)

In [ ]:
print(dataset.shape)

In [ ]:
dataset.head()

In [ ]:
sc = StandardScaler()
scale_var = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
dataset[scale_var] = sc.fit_transform(dataset[scale_var])

In [ ]:
X = dataset.drop('target', axis=1)
y = dataset.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Model Creation

*Comparing all algorithm*

In [ ]:
allAlgo = [('lr', LogisticRegression()),('knn', KNeighborsClassifier()),('dclf', DecisionTreeClassifier()),
          ('svm', SVC()),('nb', GaussianNB())]

In [ ]:
res = []
names = []
for name, algo in allAlgo:
    kfold = KFold(n_splits=10, random_state=None)
    cv_results = cross_val_score(algo, X_train, y_train, cv=kfold, scoring='accuracy')
    res.append(cv_results)
    names.append(name)
    print(name, cv_results.mean(), cv_results.std())

In [ ]:
fig = plt.figure()
plt.boxplot(res)
tick=range(1,len(names)+1)
plt.xticks(ticks=tick,labels=names)
plt.show()


###### We can identify that Logistic and SVM is giving quite good accuracy.

##### For our easiness we make a function that calculate everything with train and test

In [ ]:
def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n**********************************************")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        
    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n************************************************")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

## Logistic Regression

In [ ]:
lr = LogisticRegression(solver='saga',penalty='elasticnet',l1_ratio=0.6,max_iter=1000)
lr.fit(X_train, y_train)

print_score(lr, X_train, y_train, X_test, y_test, train=True)
print_score(lr, X_train, y_train, X_test, y_test, train=False)

## Applying PCA on Logistic Regression

In [ ]:
pca = PCA(n_components=10) 
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
expained_variance = pca.explained_variance_ratio_
classifier = LogisticRegression(random_state = 1)
classifier.fit(X_train_pca, y_train)
print_score(classifier, X_train_pca, y_train, X_test_pca, y_test, train=True)
print_score(classifier, X_train_pca, y_train, X_test_pca, y_test, train=False)

**We can observe by applying pca we are not getting a good accuracy.**

## Support Vector Machine

In [ ]:
sv = SVC(kernel='rbf',C=1)
sv.fit(X_train, y_train)
print_score(sv, X_train, y_train, X_test, y_test, train=True)
print_score(sv, X_train, y_train, X_test, y_test, train=False)

## KNeighboursClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
print_score(knn, X_train, y_train, X_test, y_test, train=True)
print_score(knn, X_train, y_train, X_test, y_test, train=False)

## GaussianNB

In [ ]:
gn = GaussianNB()
gn.fit(X_train, y_train)
print_score(gn, X_train, y_train, X_test, y_test, train=True)
print_score(gn, X_train, y_train, X_test, y_test, train=False)

## DecisionTreeClassifier

In [ ]:
dc = DecisionTreeClassifier()
dc.fit(X_train, y_train)
print_score(dc, X_train, y_train, X_test, y_test, train=True)
print_score(dc, X_train, y_train, X_test, y_test, train=False)

## Cross-validation

In [ ]:
from sklearn.model_selection import cross_val_score
cross_validation = cross_val_score(estimator = lr, X = X_train, y = y_train, cv = 10)
print("Cross validation of LR model = ",cross_validation)
print("Cross validation of LR model (in mean) = ",cross_validation.mean())


In [ ]:
from sklearn.model_selection import cross_val_score
cross_validation = cross_val_score(estimator = sv, X = X_train, y = y_train, cv = 10)
print("Cross validation of SVC model = ",cross_validation)
print("Cross validation of SVC model (in mean) = ",cross_validation.mean())

## Conclusion

**KFold Crossvalidation is performed for Logistic regression and SVM
We can conclude that logistic regression is performing well in KFold CrossValidation**

*Beginner Code Please correct if i am wrong somewhere*